In [6]:
%pip install -U langchain-ollama
%pip install customtkinter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/296.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/296.1 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/296.1 kB 445.2 kB/s eta 0:00:01
   -------------------- ------------------- 153.6/296.1 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 296.1/296.1 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import customtkinter as ctk
import tkinter as tk
from tkinter import scrolledtext
import textwrap
import json
import os

In [2]:
template = """Please list the ingredients and instructions necessary to cook this meal as though you were a living cookbook: {question} """

prompt = ChatPromptTemplate.from_template(template)

# Use the new reasoning model
model = OllamaLLM(model="hf.co/RichardErkhov/alexredna_-_TinyLlama-1.1B-Chat-v1.0-reasoning-v2-gguf:Q4_K")

# Chain the prompt and model together
chain = prompt | model

# Store all prompt-response pairs as pages
prompt_responses = []
current_page_index = 0

json_filepath = os.path.join(os.getcwd(), "recipes.json")

# Load recipes from the JSON file
def load_recipes():
    if os.path.exists(json_filepath):
        with open(json_filepath, "r") as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                return []  # Return an empty list if the file is empty or invalid
    return []

def save_recipes():
    with open(json_filepath, "w") as file:
        json.dump(prompt_responses, file, indent=4)
    print(f"Saved recipes to {os.path.abspath(json_filepath)}")

# Load recipes at startup
prompt_responses = load_recipes()

def add_page_number(page_frame, page_number):
    # Remove any existing page number labels to avoid duplication
    for widget in page_frame.winfo_children():
        if isinstance(widget, tk.Label) and widget.cget("text").startswith("Page"):
            widget.destroy()

    # Add the page number label only if content is present
    page_number_label = tk.Label(
        page_frame,
        text=f"Page {page_number}",
        font=("Times", 23, "bold"),
    )
    # Position the page number
    page_number_label.place(relx=1.0, rely=0.0001, anchor="ne")

def get_left_page_response(event=None):
    global current_page_index
    question = question_entry_left.get()
    secondary = secondary_entry_left.get()
    combined_question = f"{question}\nHere's a list of available ingredients: {secondary}"
    
    # Add the hardcoded third prompt if elaboration mode is active
    if include_third_prompt.get():
        combined_question += f"\n{hardcoded_third_prompt}"
    
    response = chain.invoke({"question": combined_question})

    # Save the question and response pair (no title needed)
    prompt_response_pair = [combined_question, response]
    
    # Update the prompt_responses list
    if len(prompt_responses) <= current_page_index:
        prompt_responses.append(prompt_response_pair)
    else:
        prompt_responses[current_page_index] = prompt_response_pair

    # Save to JSON file
    save_recipes()
    update_display()


def get_right_page_response(event=None):
    global current_page_index
    right_page_index = current_page_index + 1  # Right page is the next in sequence
    question = question_entry_right.get()
    secondary = secondary_entry_right.get()
    combined_question = f"{question}\nHere's a list of available ingredients: {secondary}"
    
    # Add the hardcoded third prompt if elaboration mode is active
    if include_third_prompt.get():
        combined_question += f"\n{hardcoded_third_prompt}"
    
    response = chain.invoke({"question": combined_question})
    
    # Save the question and response pair (no title needed)
    prompt_response_pair = [combined_question, response]
    
    # Update the prompt_responses list
    if len(prompt_responses) <= right_page_index:
        prompt_responses.append(prompt_response_pair)
    else:
        prompt_responses[right_page_index] = prompt_response_pair
    
    # Save to JSON file
    save_recipes()
    update_display()


def update_display():
    global current_page_index

    # Clear the existing page numbers
    for widget in left_page_frame.winfo_children():
        if isinstance(widget, tk.Label) and widget.cget("text").startswith("Page"):
            widget.destroy()
    for widget in right_page_frame.winfo_children():
        if isinstance(widget, tk.Label) and widget.cget("text").startswith("Page"):
            widget.destroy()

    if current_page_index == -1:
        # Reset to initial cover page format
        cover_frame.pack(fill="both", expand=True)
        book_frame.pack_forget()

        question_entry_left.delete(0, tk.END)
        secondary_entry_left.delete(0, tk.END)
        question_entry_right.delete(0, tk.END)
        secondary_entry_right.delete(0, tk.END)
        left_page_content.delete("1.0", tk.END)
        right_page_content.delete("1.0", tk.END)

        question_entry_left.insert(0, "Cookbook Field/Domain: Food")
        left_page_content.insert("1.0", "By William Johnson")

    elif current_page_index == 0:  # TOC page
        cover_frame.pack_forget()
        book_frame.pack(fill="both", expand=True)

        # Populate left page with TOC
        recipes = load_recipes()
        toc_text = "\n".join([f"{idx + 1}. {recipe[0].splitlines()[0]}" for idx, recipe in enumerate(recipes)])
        question_entry_left.delete(0, tk.END)
        secondary_entry_left.delete(0, tk.END)
        left_page_content.delete("1.0", tk.END)
        question_entry_left.insert(0, "Table of Contents")
        left_page_content.insert("1.0", toc_text)

        # Grey out TOC inputs
        question_entry_left.configure(state="disabled")
        secondary_entry_left.configure(state="disabled")
        left_page_content.configure(state="disabled")

        # Clear right page
        question_entry_right.delete(0, tk.END)
        secondary_entry_right.delete(0, tk.END)
        right_page_content.delete("1.0", tk.END)

        question_entry_right.configure(state="normal")
        secondary_entry_right.configure(state="normal")
        right_page_content.configure(state="normal")

        # Add page number for TOC
        add_page_number(left_page_frame, 0)

    else:  # Recipe pages
        # Reset to editable state for normal pages
        question_entry_left.configure(state="normal")
        secondary_entry_left.configure(state="normal")
        left_page_content.configure(state="normal")
        question_entry_right.configure(state="normal")
        secondary_entry_right.configure(state="normal")
        right_page_content.configure(state="normal")

        # Hide cover and show book view
        cover_frame.pack_forget()
        book_frame.pack(fill="both", expand=True)

        # Recipe pages index adjustment
        recipe_index = current_page_index - 1  # TOC is page 0, recipes start from page 1

        # Display content for the left page
        if 0 <= recipe_index < len(prompt_responses):
            question, response = prompt_responses[recipe_index]
            question_entry_left.delete(0, tk.END)
            question_entry_left.insert(0, question.split("\n")[0])  # Extract main question
            secondary_entry_left.delete(0, tk.END)
            secondary_entry_left.insert(0, question.split("\n")[1].replace("Here's a list of available ingredients: ", ""))
            left_page_content.delete("1.0", tk.END)
            left_page_content.insert("1.0", response)

            # Add page number for left page if it has content
            if question.strip() and response.strip():
                add_page_number(left_page_frame, recipe_index + 1)
        else:
            # Clear content for empty left page
            question_entry_left.delete(0, tk.END)
            secondary_entry_left.delete(0, tk.END)
            left_page_content.delete("1.0", tk.END)

        # Display content for the right page (next prompt and response in sequence)
        right_recipe_index = recipe_index + 1
        if 0 <= right_recipe_index < len(prompt_responses):
            question, response = prompt_responses[right_recipe_index]
            question_entry_right.delete(0, tk.END)
            question_entry_right.insert(0, question.split("\n")[0])  # Extract main question
            secondary_entry_right.delete(0, tk.END)
            secondary_entry_right.insert(0, question.split("\n")[1].replace("Here's a list of available ingredients: ", ""))
            right_page_content.delete("1.0", tk.END)
            right_page_content.insert("1.0", response)

            # Add page number for right page if it has content
            if question.strip() and response.strip():
                add_page_number(right_page_frame, right_recipe_index + 1)
        else:
            # Clear content for empty right page
            question_entry_right.delete(0, tk.END)
            secondary_entry_right.delete(0, tk.END)
            right_page_content.delete("1.0", tk.END)
            
        toc_button = ctk.CTkButton(
            left_page_frame,
            text="Table of Contents",
            command=display_toc,
        )
        toc_button.place(relx=0.9, rely=0.99, anchor="se")  # Position it to the bottom-right

def display_toc():
    global current_page_index
    current_page_index = 0  # TOC is the first logical page
    
    # Clear frames and display TOC
    cover_frame.pack_forget()
    book_frame.pack(fill="both", expand=True)
    
    # Clear the left page content
    question_entry_left.delete(0, tk.END)
    secondary_entry_left.delete(0, tk.END)
    left_page_content.delete("1.0", tk.END)

    # Set the title for the TOC page correctly
    question_entry_left.insert(0, "Table of Contents")  # Insert the title for the TOC
    
    # Generate and display the Table of Contents
    toc_text = "Table of Contents\n\n"
    for idx, entry in enumerate(prompt_responses):
        # Use the first line of the question as the title, or fallback to "Untitled"
        title = entry[0].splitlines()[0] if entry[0] else "Untitled Recipe"
        toc_text += f"{idx + 1}. {title}\n"
    
    left_page_content.insert("1.0", toc_text)

    # Disable the inputs on the TOC page
    question_entry_left.configure(state="disabled")
    secondary_entry_left.configure(state="disabled")
    left_page_content.configure(state="disabled")

    # Clear the right page for TOC
    question_entry_right.delete(0, tk.END)
    secondary_entry_right.delete(0, tk.END)
    right_page_content.delete("1.0", tk.END)
    
    question_entry_right.configure(state="disabled")
    secondary_entry_right.configure(state="disabled")
    right_page_content.configure(state="disabled")

    # Add page number for TOC (page 0)
    add_page_number(left_page_frame, 0)
    
    # **Reset the right page number to 0**
    add_page_number(right_page_frame, 0)  # Explicitly add page number for the right page


def next_page():
    global current_page_index
    if current_page_index < len(prompt_responses):  # Ensure within bounds
        current_page_index += 1
        update_display()

def previous_page():
    global current_page_index
    if current_page_index > 1:
        current_page_index -= 2
    elif current_page_index == 1:
        current_page_index = 0  # Go back to TOC
    else:
        current_page_index = -1  # Go back to cover
    update_display()

root = ctk.CTk()
root.title("Cookbook Field/Domain: Food")
root.geometry("700x500")

# Cover page setup (customtkinter)
cover_frame = ctk.CTkFrame(root)
cover_frame.pack(fill="both", expand=True)

cover_label = ctk.CTkLabel(
    cover_frame, 
    text="Cookbook Field/Domain: Food\n\nBy William Johnson", 
    font=("Times", 24), 
    justify="center", 
    padx=20, 
    pady=20
)
cover_label.pack(expand=True)

cover_button = ctk.CTkButton(
    cover_frame, 
    text="Enter Book", 
    command=update_display
)
cover_button.place(relx=0.95, rely=0.5, anchor="e")

# Hardcoded third initial prompt
hardcoded_third_prompt = "Please elaborate on the instructions of your recipe to the highest degree possible."

# Variable to track the toggle state
include_third_prompt = tk.BooleanVar(value=False)

# Function to toggle inclusion of the third prompt
def toggle_third_prompt():
    if include_third_prompt.get():
        print("Elaboration mode active.")
    else:
        print("Elaboration mode inactive.")

# Add the toggle button to the cover page
third_prompt_toggle = tk.Checkbutton(
    cover_frame,
    text="Elaboration Mode",
    variable=include_third_prompt,
    command=toggle_third_prompt
)
third_prompt_toggle.pack(side="bottom", pady=20)

def enter_book():
    global current_page_index, prompt_responses
    current_page_index = 0  # Set to TOC
    prompt_responses = load_recipes()  # Ensure recipes are loaded when entering the book
    update_display()

# Update the "Enter Book" button to use the enter_book function
cover_button.configure(command=enter_book)



# Book-style two-page display
book_frame = tk.Frame(root, relief="ridge", borderwidth=5)

# Left page setup
left_page_frame = tk.Frame(book_frame)
left_page_frame.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")
# Replace other tkinter widgets as needed
question_entry_left = ctk.CTkEntry(left_page_frame, width=400, font=("Times", 14, "bold"))
question_entry_left.bind("<Return>", get_left_page_response)
question_entry_left.pack(pady=5)

secondary_entry_left = ctk.CTkEntry(left_page_frame, width=400, font=("Times", 12))
secondary_entry_left.bind("<Return>", get_left_page_response)
secondary_entry_left.pack(pady=5)

left_page_content = scrolledtext.ScrolledText(
    left_page_frame,
    wrap="word",
    width=40,
    height=23,
    font=("Times", 12)
)
left_page_content.pack(fill="both", expand=True, padx=5, pady=(5, 10))


prev_button = ctk.CTkButton(  # Replace tk.Button with CTkButton
    left_page_frame, 
    text="Previous", 
    command=previous_page
)
prev_button.pack(side="bottom", anchor="w", pady=5)

right_page_frame = ctk.CTkFrame(book_frame)
right_page_frame.grid(row=0, column=1, padx=5, pady=5, sticky="nsew")

question_entry_right = ctk.CTkEntry(right_page_frame, width=400, font=("Times", 14, "bold"))
question_entry_right.bind("<Return>", get_right_page_response)
question_entry_right.pack(pady=5)

secondary_entry_right = ctk.CTkEntry(right_page_frame, width=400, font=("Times", 12))
secondary_entry_right.bind("<Return>", get_right_page_response)
secondary_entry_right.pack(pady=5)

right_page_content = scrolledtext.ScrolledText(
    right_page_frame,
    wrap="word",
    width=40,
    height=23,
    font=("Times", 12)
)
right_page_content.pack(fill="both", expand=True, padx=5, pady=(5, 10))

next_button = ctk.CTkButton(
    right_page_frame, 
    text="Next", 
    command=next_page
)
next_button.pack(side="bottom", anchor="e", pady=5)

book_frame.grid_columnconfigure(0, weight=1)
book_frame.grid_columnconfigure(1, weight=1)

root.mainloop()

Saved recipes to c:\Users\wdjoh\OneDrive\Desktop\recipes.json
